In [13]:
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AdapterTrainer
from datasets import load_dataset, load_from_disk
from transformers import AutoModelForQuestionAnswering
from torch.utils.data import DataLoader
from transformers import default_data_collator
import evaluate
from tqdm import tqdm
from transformers import pipeline

In [2]:
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
max_length = 512
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    context = examples["context"]
    
    # Tokenize question and context together into one input,
    # they will seperable with a special token between them.
    # Tokenizer will also split context into multiple chuncks,
    # if the max_length is exceeded.
    inputs = tokenizer(
        questions,
        context,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # offset_mapping is the result of the split into
    # multiple chunks
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    # This step is required to see if an answer is inside
    # the context chunks or not. It labels the multiple chunks
    # generated by the tokenizer into either not having
    # the answer, or where the answer is located
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [4]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    context = examples["context"]
    
    # It's the same thing as with the preprocess_training_examples
    # tokenizer, but here 
    inputs = tokenizer(
        questions,
        context,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [5]:
langs = ["en", "es", "de", "el", "ru", "tr", "ar", "vi", "zh", "hi", "ro", "th"]

In [6]:
test = load_from_disk('../data/xquad_es_test.hf/')

In [11]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [8]:
batch_size = 8
epochs = 15

In [10]:
for lang in langs:
    train = load_from_disk(f'../data/xquad_{lang}_train.hf/')
    val = load_from_disk(f'../data/xquad_{lang}_val.hf/')
    train_dataset = train.map(
        preprocess_training_examples,
        batched=True,
        remove_columns=train.column_names,
    )

    validation_dataset = val.map(
        preprocess_validation_examples,
        batched=True,
        remove_columns=val.column_names,
    )
    train_dataset.set_format("torch")
    validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
    validation_set.set_format("torch")

    train_dataloader = DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=default_data_collator,
        batch_size=8,
    )

    validation_dataloader = DataLoader(
        validation_dataset,
        shuffle=True,
        collate_fn=default_data_collator,
        batch_size=8,
    )
    adapter_name = 'squad_adapter'
    # model.add_adapter(adapter_name)
    model.load_adapter("../adapter_weights/adapter_qa_en_mbert_4ep/")
    model.train_adapter(adapter_name)
    model.set_active_adapters(adapter_name)
    model_name = model_checkpoint.split("/")[-1]
    args = TrainingArguments(
        f"{model_name}-adapter-squad",
        evaluation_strategy = "epoch",
        learning_rate=1e-4,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01
    )
    trainer = AdapterTrainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        data_collator=default_data_collator,
        tokenizer=tokenizer
    )
    trainer.train()
    model.save_adapter(f"../adapter_qa_en-{lang}_mbert_{epochs}", adapter_name=adapter_name)
    del train, val

100%|██████████| 1/1 [00:00<00:00,  2.98ba/s]
Loading cached processed dataset at ../data/xquad_en_val.hf/cache-3ca8755dd58ddcef.arrow
Loading module configuration from ../adapter_weights/adapter_qa_en_mbert_4ep/adapter_config.json
Overwriting existing adapter 'squad_adapter'.
Loading module weights from ../adapter_weights/adapter_qa_en_mbert_4ep/pytorch_adapter.bin
Loading module configuration from ../adapter_weights/adapter_qa_en_mbert_4ep/head_config.json
Loading module weights from ../adapter_weights/adapter_qa_en_mbert_4ep/pytorch_model_head.bin
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/mmm/anaconda3/envs/aml/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecat

{'eval_runtime': 22.0462, 'eval_samples_per_second': 32.976, 'eval_steps_per_second': 4.128, 'epoch': 1.0}


 13%|█▎        | 182/1365 [01:58<10:09,  1.94it/s] The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Evaluation *****
  Num examples = 727
  Batch size = 8

 13%|█▎        | 182/1365 [02:20<10:09,  1.94it/s]

{'eval_runtime': 22.1868, 'eval_samples_per_second': 32.767, 'eval_steps_per_second': 4.102, 'epoch': 2.0}


 20%|██        | 273/1365 [03:09<09:19,  1.95it/s]  The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Evaluation *****
  Num examples = 727
  Batch size = 8

 20%|██        | 273/1365 [03:31<09:19,  1.95it/s]

...

{'eval_runtime': 29.2417, 'eval_samples_per_second': 36.147, 'eval_steps_per_second': 4.548, 'epoch': 13.0}


 93%|█████████▎| 1861/1995 [21:06<01:04,  2.09it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Evaluation *****
  Num examples = 1057
  Batch size = 8

 93%|█████████▎| 1862/1995 [21:35<01:03,  2.09it/s]

{'eval_runtime': 29.2387, 'eval_samples_per_second': 36.151, 'eval_steps_per_second': 4.549, 'epoch': 14.0}


100%|█████████▉| 1994/1995 [22:38<00:00,  2.09it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Evaluation *****
  Num examples = 1057
  Batch size = 8

100%|██████████| 1995/1995 [23:08<00:00,  2.09it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1995/1995 [23:08<00:00,  1.44it/s]
Configuration saved in ../adapter_qa_en-th_mbert_15/adapter_config.json
Module weights saved in ../adapter_qa_en-th_mbert_15/pytorch_adapter.bin
Configuration saved in ../adapter_qa_en-th_mbert_15/head_config.json
Module weights saved in ../adapter_qa_en-th_mbert_15/pytorch_model_head.bin


{'eval_runtime': 29.238, 'eval_samples_per_second': 36.152, 'eval_steps_per_second': 4.549, 'epoch': 15.0}
{'train_runtime': 1388.2718, 'train_samples_per_second': 11.421, 'train_steps_per_second': 1.437, 'train_loss': 0.6782757445983122, 'epoch': 15.0}


In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
# model_checkpoint = adapter_checkpoint
question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)

context = """
? Transformers is backed by the three most popular deep learning libraries ? Jax, PyTorch and TensorFlow ? with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "What is backed by deep learning libraries?"
question_answerer(question=question, context=context)

In [8]:
def get_predictions(dataset):
    
    predictions = []
    for example in tqdm(dataset):
        question = example['question']
        context = example['context']
        prediction = question_answerer(question=question, context=context)

        predictions.append(prediction)
    
    return predictions

In [9]:
# Need to convert the variables so that they can be used by the evaluation.compute function
def convert_for_evaluation(predictions, examples):
    ref = []
    pred = []
    for i, id in enumerate(examples['id']):
        ref.append({
            'answers': examples['answers'][i],
            'id': examples['id'][i]
        })
        pred.append({
            'prediction_text': predictions[i]['answer'],
            'id': examples['id'][i]
        })
        
    return pred, ref

In [17]:
squad_metric = evaluate.load("squad")
results_en2lang = {}
results_en = {}

for lang in langs:
    test = load_from_disk(f'../data/xquad_{lang}_test.hf/')

    adapter_name = 'squad_adapter'
    model.load_adapter("../adapter_weights/adapter_qa_en_mbert_4ep/")
    model.train_adapter(adapter_name)
    model.set_active_adapters(adapter_name)

    question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)
    print('Running predictions for', lang)
    predictions = get_predictions(test)

    predictions, references = convert_for_evaluation(predictions, test) 
    res = squad_metric.compute(predictions=predictions, references=references)
    
    results_en[lang] = res
    del res
    adapter_name = 'squad_adapter'
    model.load_adapter(f"../adapter_qa_en-{lang}_mbert_15")
    model.train_adapter(adapter_name)
    model.set_active_adapters(adapter_name)

    question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)
    print(f'Running predictions for en2{lang}')
    predictions = get_predictions(test)

    predictions, references = convert_for_evaluation(predictions, test) 
    res = squad_metric.compute(predictions=predictions, references=references)
    results_en2lang[lang] = res

Overwriting existing adapter 'squad_adapter'.


Running predictions for en


  8%|▊         | 20/238 [00:05<00:52,  4.17it/s]/home/mmm/anaconda3/envs/aml/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:708: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/mmm/anaconda3/envs/aml/lib/python3.10/site-packages/transformers/pipelines/question_answering.py:298: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  p_mask = np.asarray(
100%|██████████| 238/238 [00:57<00:00,  4.14it/s]
Overwriting existing adapter 'squad_adapter'.


Running predictions for en2en


100%|██████████| 238/238 [00:56<00:00,  4.18it/s]
Overwriting existing adapter 'squad_adapter'.


...

100%|██████████| 238/238 [01:47<00:00,  2.22it/s]
Overwriting existing adapter 'squad_adapter'.


Running predictions for th


100%|██████████| 238/238 [04:17<00:00,  1.08s/it]
Overwriting existing adapter 'squad_adapter'.


Running predictions for en2th


100%|██████████| 238/238 [04:12<00:00,  1.06s/it]


In [18]:
results_en  # only using english task adapter on subset

{'en': {'exact_match': 70.16806722689076, 'f1': 83.0434082312481},
 'es': {'exact_match': 52.94117647058823, 'f1': 71.35611779333979},
 'de': {'exact_match': 47.05882352941177, 'f1': 65.20696731288513},
 'el': {'exact_match': 40.33613445378151, 'f1': 52.35982282670372},
 'ru': {'exact_match': 46.63865546218487, 'f1': 65.09564781623608},
 'tr': {'exact_match': 30.672268907563026, 'f1': 44.25426155653541},
 'ar': {'exact_match': 35.714285714285715, 'f1': 54.152242893592366},
 'vi': {'exact_match': 44.95798319327731, 'f1': 64.0526791542217},
 'zh': {'exact_match': 40.33613445378151, 'f1': 46.42256902761106},
 'hi': {'exact_match': 39.91596638655462, 'f1': 53.30651607962533},
 'ro': {'exact_match': 51.260504201680675, 'f1': 66.74265149780281},
 'th': {'exact_match': 21.84873949579832, 'f1': 29.869947979191675}}

In [19]:
results_en2lang  # using english pretrained and langspecific finetuned adapter on subset

{'en': {'exact_match': 64.28571428571429, 'f1': 77.2663503904236},
 'es': {'exact_match': 50.42016806722689, 'f1': 71.2341397809544},
 'de': {'exact_match': 50.84033613445378, 'f1': 64.97960133802835},
 'el': {'exact_match': 43.27731092436975, 'f1': 56.886697912402326},
 'ru': {'exact_match': 44.95798319327731, 'f1': 63.18026744497336},
 'tr': {'exact_match': 42.857142857142854, 'f1': 57.84173962063021},
 'ar': {'exact_match': 40.33613445378151, 'f1': 57.93892004902878},
 'vi': {'exact_match': 48.739495798319325, 'f1': 68.90732971065079},
 'zh': {'exact_match': 42.436974789915965, 'f1': 51.20915032679739},
 'hi': {'exact_match': 45.79831932773109, 'f1': 61.46453289940687},
 'ro': {'exact_match': 51.260504201680675, 'f1': 66.10247046851102},
 'th': {'exact_match': 40.7563025210084, 'f1': 46.51496962421332}}